# Time series analysis

In this notebook, we will learn how to use `rocky-worlds-utils` to generate spectral time-series and calculate a light curve from STIS and COS data.

## Imports

- `astropy`: FITS files I/O
- `matplotlib`: Plotting library
- `numpy`: array calculations
- `rocky_worlds_utils`: various functions to facilitate data analysis

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from rocky_worlds_utils.hst import time_series, stis_analysis

%matplotlib inline
pylab.rcParams['figure.figsize'] = 15.0,9.0
pylab.rcParams['font.size'] = 18

In the notebook \#2, we custom extracted data from two STIS/FUV-MAMA exposures for the target GJ 3929. Now, we shall analyze its spectral time series. For this example, we are interested in the stellar fluxes coming from the Lyman-alpha line.

The first step of this analysis is to take advantage of the `TIME-TAG` mode, which assigns a timestamp to every count that hits the detector and allows for a temporal resolution down to that of the electronics! We do not need all of that. In fact, here we shall work with a coarser temporal resolution that is more relevant for the analysis of stellar flares.

We shall use the `stis_analysis.timetag_split()` function to break down a full exposure into sub-exposures. This could take a while because in the end we need to process many subexposures.

In [ ]:
time_resolution = 60  # unit of seconds
prefix = './example_data/'
output_dir = './example_data/timeseries/'
datasets = ['ofkb08020', 'ofkb08030']

for dataset in datasets:
    stis_analysis.timetag_split(dataset, 
                                prefix, 
                                output_dir=output_dir, 
                                temporal_resolution=time_resolution, 
                                overwrite=True)

The code above should have generated `*_ts_x1d.fits` files inside the output folder (`ts` stands for time-series). Inside these new files, there are many extensions that correspond to each subexposure. We can use them to generate a light curve of the red wing of the Lyman-alpha line.

First, we need to find out which wavelength range over which we want to integrate the flux densities. Let us visually inspect the Lyman-alpha profile.

In [ ]:
y_plot_scale = 1E-14
for i, dataset in enumerate(datasets):
    spec = fits.getdata(prefix + dataset + '_x1d.fits')
    header = fits.getheader(prefix + dataset + '_x1d.fits', ext=0)
    wl = spec['WAVELENGTH'][0]
    f = spec['FLUX'][0]
    err = spec['ERROR'][0]
    plt.errorbar(wl, f / y_plot_scale, yerr=err / y_plot_scale, fmt='o', mec='white', ls='-', alpha=0.3, color='C'+str(i))
    plt.plot(wl, f / y_plot_scale, lw='2', color='C'+str(i), label=dataset)
plt.xlabel(r'Wavelength [${\rm \AA}$]')
plt.ylabel(r'Flux density [$\times 10^{%i}$ erg s$^{-1}$ cm$^{-2}$ ${\rm \AA}^{-1}$]' % int((np.log10(y_plot_scale))))
plt.xlim(1210, 1220)
plt.ylim(-0.1, 2.1)

# From a visual inspection, it seems that most of the stellar Lya flux is located within 1215.7 and 1217.5 Angstrom
plt.axvspan(xmin=1215.7, xmax=1216.7, color='C3', alpha=0.1)
# There is also some emission flux near 1218, but that is the OV line and we are not interested in it for this example.

plt.legend()
plt.tight_layout()

From a visual inspection, it seems that most of the stellar Lya flux is located within 1215.7 and 1217.5 Angstrom. So, we shall generate a light curve within this range.

In [ ]:
wavelength_range = (1215.7, 1216.7)

for dataset in datasets:
    time, flux, flux_error = time_series.generate_light_curve(
        dataset,
        prefix=output_dir,
        wavelength_range=wavelength_range
    )
    plt.errorbar(time, flux, yerr=flux_error, color='k', fmt='o')
plt.xlabel('Time (MJD)')
plt.ylabel(r'Flux (erg s$^{-1}$ cm$^{-2}$)')

As we can see in the plot above, the temporal resolution of 60 s is not very useful because it produces fluxes close to zero and with large uncertainties. Let us try a coarser temporal resolution of 300 seconds, or five minutes.

In [ ]:
time_resolution = 300  # unit of seconds
prefix = './example_data/'
output_dir = './example_data/timeseries/'
datasets = ['ofkb08020', 'ofkb08030']

for dataset in datasets:
    stis_analysis.timetag_split(dataset, 
                                prefix, 
                                output_dir=output_dir, 
                                temporal_resolution=time_resolution, 
                                overwrite=True)

In [ ]:
wavelength_range = (1215.7, 1217.5)

for dataset in datasets:
    time, flux, flux_error = time_series.generate_light_curve(
        dataset,
        prefix=output_dir,
        wavelength_range=wavelength_range
    )
    plt.errorbar(time, flux, yerr=flux_error, color='k', fmt='o')
plt.xlabel('Time (MJD)')
plt.ylabel(r'Flux (erg s$^{-1}$ cm$^{-2}$)')